In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2645)

In [15]:
x[:,:2]

matrix([[ 1233.7155907 ,  1242.33034517],
        [ 1200.35279393,  1165.75279872],
        [ 1485.72202931,  1500.8669689 ],
        [ 1244.37214432,  1163.61193635],
        [ 1508.12137715,  1496.13661074],
        [ 1676.83252923,  1602.71227194],
        [ 1283.00434033,  1285.06288384],
        [ 1294.74868919,  1296.14159982],
        [ 1719.38354082,  1770.00052896],
        [ 1568.56499889,  1519.7618474 ],
        [ 1503.05024213,  1512.86327057],
        [  956.07017215,   961.85631375],
        [ 1171.95570988,  1055.78529179],
        [ 1342.08931733,  1340.59088213],
        [ 1200.78416174,  1237.04438393],
        [ 1025.50544918,  1022.94158802],
        [ 1291.56146793,  1258.15655667],
        [ 1224.62994611,  1266.99945303],
        [ 1432.10213218,  1512.49935677],
        [ 1558.75858332,  1600.75120223],
        [ 1098.95727029,  1063.46057679],
        [ 1334.26859967,  1358.44204878],
        [ 1291.52403662,  1040.65780146],
        [ 1253.80889369,  1277.134

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 4e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.26585565e+07  0.00000000e+00  7.96e+03 1.13e+03  1.00e+06     0s
   1  -1.49070312e+07 -1.20321714e+07  9.18e+02 1.30e+02  2.06e+05     0s
   2  -9.47921794e+06 -1.09539453e+07  1.04e+01 1.47e+00  2.87e+04     0s
   3  -9.67099310e+06 -9.83963496e+06  8.84e-02 1.25e-02  3.0

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[4.287356857702462e-11,
 1285.0048896856931,
 4.967689479354947e-11,
 0.015048027516733953,
 1372.5267041074678,
 0.021566117515215644,
 1.4990924133806397e-08,
 1.0127735606927171e-05,
 4.5464708940224945e-11,
 1327.6846275934663,
 4.546470889072191e-11,
 4.5464708815118427e-11,
 4.546470885263396e-11,
 4.57254239070568e-11,
 1780.0206741204338,
 974.297971795652,
 0.03446761997269787,
 1556.1666376684107,
 0.010607040470388962,
 0.02274953666356694,
 0.01800056067856015,
 774.7102879951916,
 589.6391961299918,
 615.6347412483285,
 0.030613280902140882,
 0.028284682907963488,
 460.16027420779926,
 1242.1347989725962,
 0.010807859733044156,
 0.01443651643673588,
 0.018207553522180703,
 1261.0355279720866,
 1236.6853603281895,
 909.9208103901057,
 0.05861667169665896,
 1.3614309716640527e-06,
 1.3614487454838222e-06,
 1.3614522235660107e-06,
 0.033053606725851115,
 988.2973918349129,
 974.2552772786199,
 224.0278379667354,
 0.31084888823285084,
 2.989572499782099,
 678.2274117092279,
 0